In [483]:
import pandas as pd
import numpy as np
import re 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [484]:
#
# Read the data from CSV. Due to encoding discrepancies between UTF-8 and UTF-16 in the file
# use the 'engine="python"' parameter to read the data with proper encoding
#
journals_df = pd.read_csv('apcspend2013.csv', engine='python')

In [485]:
# 
#Check how many records are there in the dataset and how many are NULL records
#
journals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (�) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [486]:
#
# Since we are aggregating based on the 'Journal title' column we drop any record with a NULL value in that column
#
journals_df.dropna(subset=['Journal title'], inplace=True)

In [487]:
#
# Convert all the columns to upper case to get uniformity in names
#
journals_upper_case = journals_df.apply(lambda x: x.astype(str).str.upper())
#
# Renaming the long key name of cost to a shorter 'COST'
#
journals_upper_case.rename(index=str, \
                           columns={"COST (�) charged to Wellcome (inc VAT when charged)": "Cost"}, inplace=True)
#
# Remove the currency character to calculate the median and other values
#

In [488]:
#
# Create a new column called 'currency' to extract the leading currency character and convert the amount to a float
#sub = '£'
#journals_upper_case['indexes'] = journals_upper_case['Cost'].str.find(sub)
#sub = '$'
#journals_upper_case['indexes'] = journals_upper_case['Cost'].str.find(sub)
#journals_upper_case['currency'] = journals_upper_case['Cost'].astype(str).str[0]
#journals_upper_case['Cost'] = journals_upper_case['Cost'].str[1:]

In [489]:
#
# Create a new column called 'currency' to extract the leading or trainling currency character
#
journals_upper_case['currency'] = journals_upper_case['Cost'].apply(lambda x: re.findall(r'[^0-9\.]', x))

In [490]:
#
# Strip the digits only, from the Cost column
#
journals_upper_case['Cost'] = journals_upper_case['Cost'].apply\
                              (lambda x: ''.join(list(filter(str.isdigit, str(x)))))

In [491]:
#
# and convert the amount to a float
#
journals_upper_case['Cost'] = pd.to_numeric(journals_upper_case['Cost'], downcast='float')
#
# The .isdigit method extracts all digits from Cost column and ignores the decimal places. To take it to 
# original cost values the converted column must be divided by 100
journals_upper_case['Cost'] = journals_upper_case['Cost'].div(100.0).round(2)

In [492]:
# Sort the 'Journal title' column by its value
#
#journals_upper_case.sort_values("Journal title", inplace=True)

In [493]:
#
# Count the occurences of each journal title. The first 5 entries in the new array are the 5 most common journal 
# The occurence count will tells us total articles published by each journal title
#
jl_title_occurence = journals_upper_case["Journal title"].value_counts().head()
#
# Use the entries from the occurences count list and index it to our journals_upper_case array to aggregate the 
# cost and calculate median, mode and SD. 
# Make sure to check if the cost is in USD or GBP from the newly added column 'currency', use an exchange rate and
# convert the costs to a signle currency of your choice

In [499]:
# Store the key values of the journal title in a list to iterate
top_5 = jl_title_occurence.keys()

In [495]:
#
# Use the groupby function to aggregate the cost value
#
journal_title = journals_upper_case.groupby(['Journal title'])

In [522]:
#
# Create a new dataframe to calculate and store the mean, median and std. deviation of the top 5 titles
columns = ['Mean', 'Median', 'Std. Deviation']
df_final_values = pd.DataFrame(index=top_5, columns=columns)
# Convert the data type to float
df_final_values = df_final_values.apply(pd.to_numeric)

In [531]:
#
# Iterate with the keys from top_5 titles and calculate the values
for keys in top_5:
    df_final_values.at[keys, 'Mean'] = journal_title.get_group(keys).mean()
    df_final_values.at[keys, 'Median'] = journal_title.get_group(keys).median()
    df_final_values.at[keys, 'Std. Deviation'] = journal_title.get_group(keys).std()

In [532]:
df_final_values

,Mean,Median,Std. Deviation
PLOS ONE,38751.285156,898.474976,189009.515625
JOURNAL OF BIOLOGICAL CHEMISTRY,20264.634766,1314.530029,137165.500000
NEUROIMAGE,2215.168457,2326.429932,266.653961
PLOS PATHOGENS,84660.882812,1600.520020,281921.031250
PLOS GENETICS,84839.445312,1718.390015,281865.718750
